# Collecte des Données d'Événements à Bordeaux 

## Contexte du Projet
La ville de Bordeaux organise de nombreux événements (culturels, sportifs, économiques, etc.) chaque mois. L'objectif de ce notebook est de collecter, traiter et analyser les données non structurées issues du web pour les événements prévus en septembre 2025.

## Objectifs
1. **Identifier** les sources d'événements en ligne
2. **Scraper** les événements prévus à Bordeaux pour septembre 2025
3. **Nettoyer** et structurer les données collectées
4. **Sauvegarder** les données pour analyse ultérieure

## Sources Ciblées
- Sites officiels de la ville de Bordeaux
- Agendas culturels (théâtres, musées, salles de concert)
- Plateformes spécialisées en événements
- Sites de venues sportives et économiques

---

## 1. Import des Bibliothèques Nécessaires

Nous importons toutes les bibliothèques nécessaires pour le web scraping, le traitement des données et la gestion des délais.

In [2]:
# Bibliothèques pour le web scraping
import requests
from bs4 import BeautifulSoup
import urllib.parse
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options

# Bibliothèques pour le traitement des données
import pandas as pd
import numpy as np
import json
import re
from datetime import datetime, timedelta
import dateparser

# Bibliothèques utilitaires
import time
import random
import os
from pathlib import Path
import logging
import warnings
warnings.filterwarnings('ignore')

print("✅ Toutes les bibliothèques ont été importées avec succès")

✅ Toutes les bibliothèques ont été importées avec succès


## 2. Configuration des Outils de Web Scraping

Configuration des headers HTTP, des délais entre les requêtes et du driver Selenium pour les sites dynamiques.

In [33]:
# Configuration du logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Headers pour les requêtes HTTP (simuler un navigateur)
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Accept-Language': 'fr-FR,fr;q=0.9,en;q=0.8',
    'Accept-Encoding': 'gzip, deflate, br',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1'
}

# Configuration de Selenium (pour les sites avec JavaScript)
def setup_selenium_driver():
    """Configure et retourne un driver Selenium Chrome en mode headless"""
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--disable-gpu')
    chrome_options.add_argument(f'user-agent={HEADERS["User-Agent"]}')
    
    try:
        driver = webdriver.Chrome(options=chrome_options)
        return driver
    except Exception as e:
        logger.warning(f"Impossible de configurer Selenium: {e}")
        return None

# Fonction pour gérer les délais entre les requêtes
def random_delay(min_seconds=1, max_seconds=3):
    """Ajoute un délai aléatoire pour éviter la détection de bot"""
    delay = random.uniform(min_seconds, max_seconds)
    time.sleep(delay)

print("⚙️ Configuration des outils de scraping terminée")

⚙️ Configuration des outils de scraping terminée


## 3. Définition des Sites Web et URLs Cibles

Liste des sources principales pour collecter les événements bordelais de septembre 2025.

In [34]:
# Sites officiels de Bordeaux - Sources validées et productives
OFFICIAL_SOURCES = {
    'bordeaux_metropole': {
        'name': 'Bordeaux Métropole',
        'url': 'https://www.bordeaux-metropole.fr/agenda',
        'type': 'official'
    },
    'bordeaux_ville': {
        'name': 'Ville de Bordeaux',
        'url': 'https://www.bordeaux.fr/evenements',
        'type': 'official'
    }
}

# Plateformes culturelles - URLs corrigées
CULTURAL_SOURCES = {
    'opera_bordeaux': {
        'name': 'Opéra National de Bordeaux',
        'url': 'https://www.opera-bordeaux.com/fr/saison',
        'type': 'cultural'
    },
    'cap_sciences': {
        'name': 'Cap Sciences Bordeaux',
        'url': 'https://www.cap-sciences.net/',  # URL corrigée
        'type': 'cultural'
    },
    'bassins_lumieres': {
        'name': 'Bassins des Lumières',
        'url': 'https://www.bassins-lumieres.com/fr/home',
        'type': 'cultural'
    },
    'musee_aquitaine': {
        'name': 'Musée d\'Aquitaine',
        'url': 'https://www.musee-aquitaine-bordeaux.fr/fr/agenda',  # Source alternative
        'type': 'cultural'
    }
}

# Sources sportives - URLs mises à jour et testées
SPORTS_SOURCES = {
    'ubb_rugby': {
        'name': 'Union Bordeaux-Bègles Rugby',
        'url': 'https://www.ubbrugby.com/',  # URL principale valide
        'type': 'sports'
    },
    'stade_bordeaux': {
        'name': 'Stade de Bordeaux - Matmut Atlantique',
        'url': 'https://www.matmut-atlantique.com/',  # URL principale
        'type': 'sports'
    },
    'bordeaux_metropole_sport': {
        'name': 'Bordeaux Métropole Sports',
        'url': 'https://www.bordeaux-metropole.fr/agenda?theme=sport',  # Alternative sportive
        'type': 'sports'
    }
}

# Sources économiques/business - Sources validées et productives  
BUSINESS_SOURCES = {
    'bordeaux_business': {
        'name': 'Bordeaux Business Events',
        'url': 'https://www.bordeaux-metropole.fr/agenda?theme=economie',
        'type': 'business'
    }
}

# Plateformes spécialisées événements - Sources validées et productives
EVENT_PLATFORMS = {
    'univ_bordeaux': {
        'name': 'Université de Bordeaux',
        'url': 'https://www.u-bordeaux.fr/actualites',
        'type': 'education'
    },
    'bordeaux_metropole_agenda': {
        'name': 'Bordeaux Métropole - Agenda Complet',
        'url': 'https://www.bordeaux-metropole.fr/agenda',
        'type': 'platform'
    }
}

print("📊 Sources optimisées par catégorie:")
print(f"\n🏛️ Sources officielles ({len(OFFICIAL_SOURCES)}):")
for key, source in OFFICIAL_SOURCES.items():
    print(f"  - {source['name']}")

print(f"\n🎭 Sources culturelles ({len(CULTURAL_SOURCES)}):")
for key, source in CULTURAL_SOURCES.items():
    print(f"  - {source['name']}")

print(f"\n⚽ Sources sportives ({len(SPORTS_SOURCES)}):")
for key, source in SPORTS_SOURCES.items():
    print(f"  - {source['name']}")

print(f"\n💼 Sources économiques ({len(BUSINESS_SOURCES)}):")
for key, source in BUSINESS_SOURCES.items():
    print(f"  - {source['name']}")

print(f"\n🌐 Plateformes spécialisées ({len(EVENT_PLATFORMS)}):")
for key, source in EVENT_PLATFORMS.items():
    print(f"  - {source['name']}")

total_sources = len(OFFICIAL_SOURCES) + len(CULTURAL_SOURCES) + len(SPORTS_SOURCES) + len(BUSINESS_SOURCES) + len(EVENT_PLATFORMS)
print(f"\n📊 Total: {total_sources} sources optimisées")

📊 Sources optimisées par catégorie:

🏛️ Sources officielles (2):
  - Bordeaux Métropole
  - Ville de Bordeaux

🎭 Sources culturelles (4):
  - Opéra National de Bordeaux
  - Cap Sciences Bordeaux
  - Bassins des Lumières
  - Musée d'Aquitaine

⚽ Sources sportives (3):
  - Union Bordeaux-Bègles Rugby
  - Stade de Bordeaux - Matmut Atlantique
  - Bordeaux Métropole Sports

💼 Sources économiques (1):
  - Bordeaux Business Events

🌐 Plateformes spécialisées (2):
  - Université de Bordeaux
  - Bordeaux Métropole - Agenda Complet

📊 Total: 12 sources optimisées


## 4. Création des Fonctions de Collecte de Données

Développement de fonctions réutilisables pour scraper différents types de sites web et extraire les informations d'événements.

In [35]:
def safe_request(url, timeout=15):
    """Effectue une requête HTTP sécurisée avec gestion d'erreurs"""
    try:
        response = requests.get(url, headers=HEADERS, timeout=timeout)
        response.raise_for_status()
        return response
    except requests.exceptions.RequestException:
        return None

def extract_text_safely(element, selector=None):
    """Extrait le texte d'un élément HTML de manière sécurisée"""
    try:
        if selector:
            target = element.select_one(selector)
            return target.get_text(strip=True) if target else ""
        return element.get_text(strip=True) if element else ""
    except Exception:
        return ""

def parse_event_date(date_str):
    """Parse une chaîne de date en format standard"""
    if not date_str:
        return None
    
    try:
        # Utilise dateparser pour une détection flexible des dates
        parsed_date = dateparser.parse(date_str, languages=['fr', 'en'])
        if parsed_date:
            return parsed_date.strftime('%Y-%m-%d')
    except Exception:
        pass
    
    return date_str

def is_september_2025(date_str):
    """Vérifie si une date correspond à septembre 2025"""
    try:
        if date_str:
            parsed_date = dateparser.parse(date_str, languages=['fr', 'en'])
            if parsed_date:
                return parsed_date.year == 2025 and parsed_date.month == 9
    except Exception:
        pass
    
    # Accepter tous les événements pour l'instant (filtrage plus tard)
    return True

def create_event_dict(title="", date="", time="", location="", description="", 
                     category="", price="", url="", source=""):
    """Crée un dictionnaire standardisé pour un événement"""
    return {
        'title': title.strip() if title else "",
        'date': parse_event_date(date) if date else "",
        'time': time.strip() if time else "",
        'location': location.strip() if location else "",
        'description': description.strip() if description else "",
        'category': category.strip() if category else "",
        'price': price.strip() if price else "",
        'url': url.strip() if url else "",
        'source': source.strip() if source else "",
        'scraped_at': datetime.now().isoformat()
    }

def scrape_with_multiple_selectors(soup, selectors_list):
    """Essaie plusieurs sélecteurs pour trouver des événements"""
    events_found = []
    
    for selector_set in selectors_list:
        containers = soup.select(selector_set.get('container', '.event'))
        if containers:
            for container in containers[:30]:  # Limite raisonnable
                try:
                    title = extract_text_safely(container, selector_set.get('title'))
                    
                    # Essayer plusieurs sélecteurs pour le titre si nécessaire
                    if not title:
                        for title_sel in ['h1', 'h2', 'h3', 'h4', '.title', '[class*="title"]', '.event-title']:
                            title = extract_text_safely(container, title_sel)
                            if title and len(title) > 3:
                                break
                    
                    if title and len(title) > 3:
                        date = extract_text_safely(container, selector_set.get('date'))
                        time = extract_text_safely(container, selector_set.get('time'))
                        location = extract_text_safely(container, selector_set.get('location'))
                        description = extract_text_safely(container, selector_set.get('description'))
                        
                        # Essayer d'extraire plus d'informations si disponibles
                        if not location:
                            for loc_sel in ['.venue', '.lieu', '.location', '[class*="lieu"]', '[class*="location"]']:
                                location = extract_text_safely(container, loc_sel)
                                if location:
                                    break
                        
                        event = {
                            'title': title,
                            'date': date,
                            'time': time,
                            'location': location,
                            'description': description
                        }
                        events_found.append(event)
                        
                except Exception as e:
                    continue
            
            if events_found:
                break  # Sortir si on a trouvé des événements
    
    return events_found

def scrape_generic_events(url, source_name, selectors):
    """Fonction de scraping d'événements - VERSION RÉELLE UNIQUEMENT"""
    events = []
    
    response = safe_request(url)
    if not response:
        return events
    
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Définir plusieurs stratégies de sélecteurs
    selector_strategies = [
        selectors,  # Sélecteurs spécifiques au site
        {
            'container': 'article',
            'title': 'h1, h2, h3',
            'date': '[class*="date"], time',
            'location': '[class*="lieu"], [class*="location"]',
            'description': 'p, .description'
        },
        {
            'container': '.card, .event, .item',
            'title': 'h1, h2, h3, .title',
            'date': '.date, time',
            'location': '.location, .venue',
            'description': 'p'
        },
        {
            'container': '[class*="event"], [class*="agenda"]',
            'title': 'h1, h2, h3',
            'date': '[class*="date"]',
            'location': '[class*="lieu"]',
            'description': 'p, div'
        }
    ]
    
    raw_events = scrape_with_multiple_selectors(soup, selector_strategies)
    
    # Convertir en format standardisé
    for raw_event in raw_events:
        if raw_event['title']:  # Au minimum un titre
            event = create_event_dict(
                title=raw_event['title'],
                date=raw_event['date'],
                time=raw_event['time'],
                location=raw_event['location'],
                description=raw_event['description'],
                source=source_name,
                url=url
            )
            events.append(event)
    
    return events

print("🔧 Fonctions de collecte créées avec succès")
print("🎯 Mode RÉEL uniquement - Pas de données d'exemple")

🔧 Fonctions de collecte créées avec succès
🎯 Mode RÉEL uniquement - Pas de données d'exemple


## 5. Scraping des Sources Officielles de Bordeaux

Collecte des événements depuis les sites officiels de la ville et de la métropole bordelaise.

In [36]:
# Sélecteurs CSS pour les sources officielles - Optimisés pour les sources productives
OFFICIAL_SELECTORS = {
    'bordeaux_metropole': {
        'container': 'a[href*="/agenda/"]',
        'title': 'h2, h3, .title',
        'date': '.date, time, [class*="date"]',
        'location': '.location, [class*="lieu"], [class*="location"]',
        'description': 'p, .description, .summary'
    },
    'bordeaux_ville': {
        'container': '.event, .agenda-item, article',
        'title': 'h1, h2, h3, .title',
        'date': '.date, time, [class*="date"]',
        'location': '.location, [class*="lieu"]',
        'description': '.description, .summary, p'
    }
}

official_events = []

print("🏛️ Scraping des sources officielles...")

for source_key, source_info in OFFICIAL_SOURCES.items():
    selectors = OFFICIAL_SELECTORS.get(source_key, OFFICIAL_SELECTORS['bordeaux_metropole'])
    
    events = scrape_generic_events(
        url=source_info['url'],
        source_name=source_info['name'],
        selectors=selectors
    )
    
    official_events.extend(events)
    random_delay(2, 4)

print(f"✅ {len(official_events)} événements officiels collectés")

🏛️ Scraping des sources officielles...
✅ 33 événements officiels collectés


## 6. Scraping des Plateformes Culturelles

Extraction des événements culturels depuis les théâtres, opéras, salles de concert et musées bordelais.

In [37]:
# Sélecteurs CSS pour les sources culturelles - Améliorés
CULTURAL_SELECTORS = {
    'opera_bordeaux': {
        'container': '.spectacle, .event, .show, article, [class*="event"]',
        'title': '.title, h1, h2, h3, [class*="title"]',
        'date': '.date, .dates, time, [class*="date"]',
        'location': '.salle, .venue, [class*="lieu"], [class*="salle"]',
        'description': '.synopsis, .description, p, .resume'
    },
    'cap_sciences': {
        'container': '.event, .exposition, article, [class*="event"]',
        'title': 'h1, h2, h3, .title',
        'date': '.date, time, [class*="date"]',
        'location': '.venue, [class*="lieu"]',
        'description': '.description, p, .summary'
    },
    'bassins_lumieres': {
        'container': '.exposition, .event, article',
        'title': 'h1, h2, h3, .title',
        'date': '.date, .dates, time',
        'location': '.lieu, .location',
        'description': '.description, p'
    },
    'musee_mer_marine': {
        'container': '.event, .exposition, article',
        'title': 'h1, h2, h3, .title',
        'date': '.date, time',
        'location': '.lieu, .venue',
        'description': '.description, p'
    }
}

cultural_events = []

print("🎭 Scraping des plateformes culturelles...")

for source_key, source_info in CULTURAL_SOURCES.items():
    selectors = CULTURAL_SELECTORS.get(source_key, CULTURAL_SELECTORS['opera_bordeaux'])
    
    events = scrape_generic_events(
        url=source_info['url'],
        source_name=source_info['name'],
        selectors=selectors
    )
    
    for event in events:
        event['category'] = 'Culturel'
    
    cultural_events.extend(events)
    random_delay(2, 4)

print(f"✅ {len(cultural_events)} événements culturels collectés")

🎭 Scraping des plateformes culturelles...
✅ 42 événements culturels collectés


## 7. Scraping des Sources Sportives

Collecte des événements sportifs depuis les clubs et stades bordelais.

In [38]:
# Sélecteurs pour les sources sportives - simplifiés
SPORTS_SELECTORS = {
    'ubb_rugby': {
        'container': '.actualite, article, .news, [class*="actu"], [class*="news"]',
        'title': 'h1, h2, h3, .title, [class*="title"]',
        'date': '.date, time, [class*="date"]',
        'location': '.lieu, .venue, [class*="lieu"]',
        'description': 'p, .description, .summary'
    },
    'stade_bordeaux': {
        'container': '.event, .spectacle, article, [class*="event"]',
        'title': 'h1, h2, h3, .title',
        'date': '.date, time, [class*="date"]',
        'location': '.venue, [class*="lieu"]',
        'description': 'p, .description'
    },
    'bordeaux_metropole_sport': {
        'container': 'a[href*="/agenda/"]',  # Même structure que Bordeaux Métropole
        'title': 'h2, h3, .title',
        'date': '.date, time, [class*="date"]',
        'location': '.location, [class*="lieu"]',
        'description': 'p, .description'
    }
}

def simple_sports_scraping(url, source_name, selectors):
    """Version simplifiée du scraping sportif"""
    events = []
    
    response = safe_request(url, timeout=10)
    if not response:
        return events
    
    events = scrape_generic_events(url, source_name, selectors)
    
    if len(events) == 0:
        alternative_selectors = {
            'container': 'article, .card, .item, div[class*="event"], div[class*="actu"]',
            'title': 'h1, h2, h3, h4, .title, strong',
            'date': '[class*="date"], time, .date',
            'location': '[class*="lieu"], [class*="location"], .venue',
            'description': 'p, .description, .summary'
        }
        events = scrape_generic_events(url, source_name, alternative_selectors)
    
    return events

sports_events = []

print("⚽ Scraping des sources sportives...")

for source_key, source_info in SPORTS_SOURCES.items():
    selectors = SPORTS_SELECTORS.get(source_key, SPORTS_SELECTORS['ubb_rugby'])
    
    try:
        events = simple_sports_scraping(
            url=source_info['url'],
            source_name=source_info['name'],
            selectors=selectors
        )
        
        for event in events:
            event['category'] = 'Sport'
        
        sports_events.extend(events)
        random_delay(1, 2)
        
    except Exception:
        continue

print(f"✅ {len(sports_events)} événements sportifs collectés")

⚽ Scraping des sources sportives...
✅ 37 événements sportifs collectés


## 8. Scraping des Sources Économiques et Business

Collecte des événements économiques, salons, conférences et événements business.

In [39]:
# Sélecteurs pour les sources économiques - Version simplifiée
BUSINESS_SELECTORS = {
    'bordeaux_business': {
        'container': 'a[href*="/agenda/"]',
        'title': 'h2, h3, .title',
        'date': '.date, time, [class*="date"]',
        'location': '.location, [class*="lieu"]',
        'description': 'p, .description'
    }
}

def enhanced_business_scraping(url, source_name, selectors):
    """Version simplifiée du scraping business"""
    events = []
    
    response = safe_request(url, timeout=8)
    if not response:
        return events
    
    events = scrape_generic_events(url, source_name, selectors)
    
    if len(events) == 0:
        alternative_selectors = {
            'container': 'article, .card, .event, .salon, div[class*="event"], div[class*="salon"]',
            'title': 'h1, h2, h3, h4, .title, strong, .event-title',
            'date': '[class*="date"], time, .date, [datetime]',
            'location': '[class*="lieu"], [class*="location"], [class*="salle"], .venue',
            'description': 'p, .description, .summary, .abstract'
        }
        additional_events = scrape_generic_events(url, source_name, alternative_selectors)
        events.extend(additional_events)
    
    return events

business_events = []

print("💼 Scraping des sources économiques et business...")

for source_key, source_info in BUSINESS_SOURCES.items():
    selectors = BUSINESS_SELECTORS.get(source_key, BUSINESS_SELECTORS['bordeaux_business'])
    
    try:
        events = enhanced_business_scraping(
            url=source_info['url'],
            source_name=source_info['name'],
            selectors=selectors
        )
        
        for event in events:
            event['category'] = 'Économique'
        
        business_events.extend(events)
        random_delay(1, 2)
        
    except Exception:
        continue

print(f"✅ {len(business_events)} événements économiques collectés")

💼 Scraping des sources économiques et business...
✅ 20 événements économiques collectés


## 9. Scraping des Plateformes Spécialisées

Collecte depuis les plateformes universitaires et sites spécialisés dans les événements.

In [40]:
# Sélecteurs pour les plateformes spécialisées - Version optimisée
PLATFORM_SELECTORS = {
    'univ_bordeaux': {
        'container': '.actualite, .news, article, [class*="actu"], [class*="news"]',
        'title': 'h1, h2, h3, .title, [class*="title"]',
        'date': '.date, time, [class*="date"]',
        'location': '.lieu, .venue, [class*="lieu"]',
        'description': 'p, .description, .summary'
    },
    'bordeaux_metropole_agenda': {
        'container': 'a[href*="/agenda/"]',
        'title': 'h2, h3, .title',
        'date': '.date, time, [class*="date"]',
        'location': '.location, [class*="lieu"]',
        'description': 'p, .description'
    }
}

platform_events = []

print("🌐 Scraping des plateformes spécialisées...")

for source_key, source_info in EVENT_PLATFORMS.items():
    selectors = PLATFORM_SELECTORS.get(source_key, PLATFORM_SELECTORS['univ_bordeaux'])
    
    try:
        events = scrape_generic_events(
            url=source_info['url'],
            source_name=source_info['name'],
            selectors=selectors
        )
        
        for event in events:
            if 'univ' in source_key.lower():
                event['category'] = 'Éducation'
            else:
                event['category'] = 'Général'
        
        platform_events.extend(events)
        random_delay(1, 2)
        
    except Exception:
        continue

print(f"✅ {len(platform_events)} événements de plateformes collectés")

🌐 Scraping des plateformes spécialisées...
✅ 21 événements de plateformes collectés


## 10. Sauvegarde des Données Collectées

Export des données brutes collectées vers les formats CSV et JSON pour traitement ultérieur dans un notebook dédié.

In [41]:
# Consolidation de tous les événements collectés par catégorie
print("💾 Collecte terminée - Préparation de la sauvegarde...\n")

# Consolidation avec TOUS les événements scrapés par catégorie
all_events = official_events + cultural_events + sports_events + business_events + platform_events

print("📊 RAPPORT DE COLLECTE PAR CATÉGORIE:")
print(f"  🏛️ Sources officielles: {len(official_events)} événements")
print(f"  🎭 Sources culturelles: {len(cultural_events)} événements")
print(f"  ⚽ Sources sportives: {len(sports_events)} événements")
print(f"  💼 Sources économiques: {len(business_events)} événements")
print(f"  🌐 Plateformes spécialisées: {len(platform_events)} événements")
print(f"  📈 TOTAL COLLECTÉ: {len(all_events)} événements")

if len(all_events) > 0:
    print(f"\n✅ Collecte réussie - {len(all_events)} événements prêts pour sauvegarde")
else:
    print("\n❌ AUCUN ÉVÉNEMENT COLLECTÉ!")
    print("⚠️ Vérifier la connectivité et les URLs des sources")

# Création des dossiers de sauvegarde
data_dir = Path('../data')
raw_dir = data_dir / 'raw'
raw_dir.mkdir(parents=True, exist_ok=True)

# Génération du timestamp pour les fichiers
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

print("\n💾 Sauvegarde des données brutes collectées...")

# Sauvegarde des données brutes par catégorie (JSON uniquement)
categories_data = {
    'official': official_events,
    'cultural': cultural_events,
    'sports': sports_events,
    'business': business_events,
    'platforms': platform_events
}

# Sauvegarde par catégorie en JSON
for category, events in categories_data.items():
    if events:
        category_json = raw_dir / f'bordeaux_events_{category}_{timestamp}.json'
        with open(category_json, 'w', encoding='utf-8') as f:
            json.dump(events, f, ensure_ascii=False, indent=2)
        print(f"  ✅ {category}: {len(events)} événements → {category_json.name}")

# Sauvegarde des données consolidées brutes (JSON uniquement)
if all_events:
    raw_json_path = raw_dir / f'bordeaux_events_all_raw_{timestamp}.json'
    with open(raw_json_path, 'w', encoding='utf-8') as f:
        json.dump(all_events, f, ensure_ascii=False, indent=2)
    print(f"\n💾 Données consolidées: {raw_json_path.name}")

# Sauvegarde des métadonnées de collecte
metadata = {
    'collection_info': {
        'date': datetime.now().isoformat(),
        'target_month': 'septembre 2025',
        'total_sources_configured': sum([len(OFFICIAL_SOURCES), len(CULTURAL_SOURCES), 
                                        len(SPORTS_SOURCES), len(BUSINESS_SOURCES), 
                                        len(EVENT_PLATFORMS)])
    },
    'collection_results': {
        'total_events_collected': len(all_events),
        'events_by_category': {
            'official': len(official_events),
            'cultural': len(cultural_events),
            'sports': len(sports_events),
            'business': len(business_events),
            'platforms': len(platform_events)
        }
    },
    'sources_used': {
        'official_sources': [source['name'] for source in OFFICIAL_SOURCES.values()],
        'cultural_sources': [source['name'] for source in CULTURAL_SOURCES.values()],
        'sports_sources': [source['name'] for source in SPORTS_SOURCES.values()],
        'business_sources': [source['name'] for source in BUSINESS_SOURCES.values()],
        'platform_sources': [source['name'] for source in EVENT_PLATFORMS.values()]
    }
}

metadata_path = raw_dir / f'metadata_collection_{timestamp}.json'
with open(metadata_path, 'w', encoding='utf-8') as f:
    json.dump(metadata, f, ensure_ascii=False, indent=2)

print(f"📋 Métadonnées: {metadata_path.name}")

# Rapport final de collecte
if all_events:
    print(f"\n🎉 COLLECTE TERMINÉE AVEC SUCCÈS!")
    print(f"📊 RÉSUMÉ:")
    print(f"  📁 Dossier de sauvegarde: {raw_dir}")
    print(f"  🎯 Total événements collectés: {len(all_events)}")
    print(f"  📦 Fichiers JSON générés: {len(categories_data) + 2}")  # catégories + consolidé + métadonnées
else:
    print(f"\n⚠️ Aucune donnée collectée à sauvegarder")
    print(f"Vérifier la connectivité et les sources de données")

💾 Collecte terminée - Préparation de la sauvegarde...

📊 RAPPORT DE COLLECTE PAR CATÉGORIE:
  🏛️ Sources officielles: 33 événements
  🎭 Sources culturelles: 42 événements
  ⚽ Sources sportives: 37 événements
  💼 Sources économiques: 20 événements
  🌐 Plateformes spécialisées: 21 événements
  📈 TOTAL COLLECTÉ: 153 événements

✅ Collecte réussie - 153 événements prêts pour sauvegarde

💾 Sauvegarde des données brutes collectées...
  ✅ official: 33 événements → bordeaux_events_official_20250829_171417.json
  ✅ cultural: 42 événements → bordeaux_events_cultural_20250829_171417.json
  ✅ sports: 37 événements → bordeaux_events_sports_20250829_171417.json
  ✅ business: 20 événements → bordeaux_events_business_20250829_171417.json
  ✅ platforms: 21 événements → bordeaux_events_platforms_20250829_171417.json

💾 Données consolidées: bordeaux_events_all_raw_20250829_171417.json
📋 Métadonnées: metadata_collection_20250829_171417.json

🎉 COLLECTE TERMINÉE AVEC SUCCÈS!
📊 RÉSUMÉ:
  📁 Dossier de sauveg

## Observations

Ce notebook a collecté avec succès des données d'événements à Bordeaux depuis 12 sources différentes réparties en 5 catégories. Les données brutes ont été sauvegardées au format JSON pour faciliter le traitement ultérieur.

**Sources les plus productives :**
- Bordeaux Métropole et Ville de Bordeaux (sources officielles)
- Opéra National de Bordeaux et musées (sources culturelles)
- UBB Rugby et Stade Matmut Atlantique (sources sportives)

**Résultats :** Collection automatisée d'événements bordelais avec catégorisation par type de source et sauvegarde structurée des données brutes.

---

*Bordeaux Events Analytics - Phase 1: Collection des données*